<a href="https://colab.research.google.com/github/SpandanaKalakonda/Building_ML_Pipelines/blob/main/counting_words_using_apache_beam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install apache-beam

In [7]:
#Import neccessary libraries
import re
import apache_beam as beam
from apache_beam.io import ReadFromText
from apache_beam.io import WriteToText
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import SetupOptions

In [8]:
# Specify the input and putput file paths
input_file = "/content/kinglear.txt"
output_file = "/content/output.txt"

In [9]:
# Create a pipelie options object
pipeline_options = PipelineOptions()

In [10]:
with beam.Pipeline(options = pipeline_options) as p:

  # Read the input file bytes into the PCollection
  lines = p | beam.io.ReadFromText(input_file , coder=beam.coders.BytesCoder())

  # Decode the byte data to string with utf-8 encoding
  decoded_lines = lines | beam.Map(lambda line: line.decode('utf-8'))

  # Count the occurances of each word in the input file
  counts = (
    decoded_lines
    | 'Split' >> beam.FlatMap(lambda x: re.findall(r'[A-Za-z\']+', x))
    | 'PairWithOne' >> beam.Map(lambda x: (x,1))
    | 'GroupAndSum' >> beam.CombinePerKey(sum)
    )

  # Format the result of counts into a PCollection of Strings
  def format_result(word_count):
    """Convert tuples (token, count) into a string"""
    (word, count) = word_count
    return "{}: {}".format(word, count)

  output = counts | 'Format' >> beam.Map(format_result)

  # Write the output to a outputfile
  output | WriteToText(output_file)
